In [163]:
import nfl_regression
import pandas as pd

## Importing the dataset

In [164]:
train_receiver_stats, train_player_stats, train_weeks = nfl_regression.preprocess_year_data('2021')
X_train, Y_train = nfl_regression.get_yards_by_stats(train_receiver_stats, train_player_stats, train_weeks, '2021')

test_receiver_stats, test_player_stats, test_weeks = nfl_regression.preprocess_year_data('2023')
X_test, Y_test = nfl_regression.get_yards_by_stats(test_receiver_stats, test_player_stats, test_weeks, '2023')

## Training the SVM

In [165]:
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest, f_regression
reg = SVR(kernel='poly')
k_best = SelectKBest(f_regression, k=15).fit(X_train, Y_train)
Xk_train, Xk_test = k_best.transform(X_train), k_best.transform(X_test)
reg = reg.fit(Xk_train, Y_train)
print(f'train {reg.score(Xk_train, Y_train):.4f}', f'test {reg.score(Xk_test, Y_test):.4f}')

train 0.5865 test 0.5056


## Importing play data

In [166]:
game_data = pd.read_csv('BigDataBowl2023\data2023\games.csv')
game_data.head(1)

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL


In [167]:
play_data = pd.read_csv('BigDataBowl2023\data2023\plays.csv')
play_data.head(1)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man


## Load week data into single DataFrame

In [168]:
week_data = pd.concat([pd.read_csv(f'BigDataBowl2023\data2023\week{i}.csv') for i in range(1, 9)])

In [169]:
week_data.head(1)

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.3,0.03,165.16,84.99,None


## DataFrame of offensive players on pitch

In [170]:
pitch_lookup = pd.concat((play_data[['gameId', 'playId']], pd.DataFrame(play_data.apply(lambda x: week_data[(week_data['gameId'] == x['gameId']) & (week_data['playId'] == x['playId'])][week_data[(week_data['gameId'] == x['gameId']) & (week_data['playId'] == x['playId'])]['team'] == x['possessionTeam']]['nflId'].unique(), axis=1), columns=['players'])), axis=1)
pitch_lookup.to_csv('BigDataBowl2023/pitch_lookup.csv', index=False)